# INF-510, Claudio Torres, claudio.torres@usm.cl, DI-UTFSM
## Textbook: Lloyd N. Trefethen, Spectral Methods in MATLAB, SIAM, Philadelphia, 2000

# More on Spectral Matrices

In [89]:
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import scipy as sp # type: ignore
import sympy as sym # type: ignore
from scipy.linalg import toeplitz # type: ignore

from ipywidgets import IntSlider # type: ignore
from matplotlib import cm # type: ignore
# The variable M is used for changing the default size of the figures
M=5

from ipywidgets import interact # type: ignore

FS = 20
plt.rcParams.update({
    'font.size': FS,
    'text.usetex': True,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',
    'xtick.labelsize': FS,
    'ytick.labelsize': FS,
    'axes.labelsize': FS,
    'text.latex.preamble': r'\usepackage{amsfonts}'
})

sym.init_printing()

### Chebyshev differentiation matrix

In [90]:
def cheb(N):
    if N==0:
        D=0
        x=1
        return D,x
    x  = np.cos(np.pi*np.arange(N+1)/N)
    c  = np.hstack((2,np.ones(N-1),2))*((-1.)**np.arange(N+1))
    X  = np.tile(x,(N+1,1)).T
    dX = X-X.T
    D  = np.outer(c,1./c)/(dX+np.eye(N+1))
    D  = D - np.diag(np.sum(D.T,axis=0))
    return D,x

## Understanding how the np.FFT does the FFT

In [91]:
def show_spectral_derivative_example(N):
    x=np.linspace(2*np.pi/N,2*np.pi,N)
    u = lambda x: np.sin(x)
    up = lambda x: np.cos(x)
        
    v=u(x)
    
    # Computing the spectral derivative using FFT
    K=np.fft.fftfreq(N)*N
    iK=1j*K
    vhat=np.fft.fft(v)
    W=iK*vhat
    W[int(N/2)]=0
    vp=np.real(np.fft.ifft(W))

    plt.figure(figsize=(10,10))
    plt.plot(x,v,'ks-',markersize=12,markeredgewidth=3,label=r'$\sin(x)$',linewidth=3)
    plt.plot(x,up(x),'b.-',markersize=24,markeredgewidth=3,label=r'$\mathrm{Exact\ derivative:} \cos(x)$',linewidth=3)
    plt.plot(x,np.real(vp),'rx-',markersize=10,markeredgewidth=3,label=r'$\mathrm{Spectral\ derivative}$',linewidth=3)
    plt.grid(True)
    plt.legend(loc='lower left')
    plt.xlabel('$x$')
    plt.show()
    
    print('v    :',v)
    print('vhat :',vhat)
    print('K    :',K)
    print('W    :',W)
    print('vprime: ',vp)
interact(show_spectral_derivative_example,N=(2,40,2))

interactive(children=(IntSlider(value=20, description='N', max=40, min=2, step=2), Output()), _dom_classes=('w…

<function __main__.show_spectral_derivative_example(N)>

In [92]:
def spectralDerivativeByFFT(v,nu=1):
    if not np.all(np.isreal(v)):
        raise ValueError('The input vector must be real')
    N=v.shape[0]
    K=np.fft.fftfreq(N)*N
    iK=(1j*K)**nu
    v_hat=np.fft.fft(v)
    w_hat=iK*v_hat
    if np.mod(nu,2)!=0:
        w_hat[int(N/2)]=0
    return np.real(np.fft.ifft(w_hat))

def my_D2_spec_2pi(N):
    h=(2*np.pi/N)
    c=np.zeros(N)
    j=np.arange(1,N)
    c[0]=-np.pi**2/(3.*h**2)-1./6.
    c[1:]=-0.5*((-1)**j)/(np.sin(j*h/2.)**2)
    D2=toeplitz(c)
    return D2

# Fractional derivative application

In [93]:
def fractional_derivative(N=10,nu=1):
    x=np.linspace(2*np.pi/N,2*np.pi,N)
    u = lambda x: np.sin(x)
    up = lambda x: np.cos(x)
    v = u(x)
    vp=spectralDerivativeByFFT(v,nu)
    plt.figure(figsize=(10,10))
    plt.plot(x,v,'ks-',markersize=12,markeredgewidth=3,label=r'$\sin(x)$',linewidth=3)
    plt.plot(x,up(x),'b.-',markersize=24,markeredgewidth=3,label=r'$\mathrm{Exact\ derivative:} \cos(x)$',linewidth=3)
    plt.plot(x,np.real(vp),'rx-',markersize=10,markeredgewidth=3,label=r'$\frac{d^{\nu}u}{dx^{\nu}}$',linewidth=3)
    plt.grid(True)
    plt.legend(loc='lower left')
    plt.xlabel('$x$')
    plt.show()
d_nu=0.1
interact(fractional_derivative,N=(4,100),nu=(0.0,4,d_nu))

interactive(children=(IntSlider(value=10, description='N', min=4), FloatSlider(value=1.0, description='nu', ma…

<function __main__.fractional_derivative(N=10, nu=1)>

# Example 1: Computing Eigenvalues

We are solving: $-u''(x)+x^2\,u(x)=\lambda\, u(x)$ on $\mathbb{R}$

In [94]:
L=8.0
def show_example_1(N=6):
    h=2*np.pi/N
    x=np.linspace(h,2*np.pi,N)
    x=L*(x-np.pi)/np.pi
    D2=(np.pi/L)**2*my_D2_spec_2pi(N)
    w, v = np.linalg.eig(-D2+np.diag(x**2))
    ii = np.argsort(w)
    w=w[ii]
    v=v[:,ii]
    
    plt.figure(figsize=(2*M,2*M))

    for i in np.arange(1,5):
        plt.subplot(2,2,i)
        plt.title(r'$u_{:d}(x),\, \lambda_{:d}={:f}$'.format(i,i,w[i-1]))
        plt.plot(x,v[:,i],'kx',markersize=16,markeredgewidth=3)
        plt.grid(True)
    plt.show()
interact(show_example_1,N=(6,100,1))

interactive(children=(IntSlider(value=6, description='N', min=6), Output()), _dom_classes=('widget-interact',)…

<function __main__.show_example_1(N=6)>

# Example 2:  Solving ODE

Solving the following BVP $u_{xx}=\exp(4\,x)$ with $u(-1)=u(1)=0$

In [95]:
def example_2(N=16):
    D,x = cheb(N)
    D2  = np.dot(D,D)
    D2  = D2[1:-1,1:-1]
    f   = np.exp(4*x[1:-1])
    u   = np.linalg.solve(D2,f)
    u   = np.concatenate(([0],u,[0]),axis=0)

    plt.figure(figsize=(M,M))
    plt.plot(x,u,'k.')
    xx  = np.linspace(-1,1,1000)
    P   = np.polyfit(x, u, N)
    uu  = np.polyval(P, xx)
    plt.plot(xx,uu,'b-')
    plt.grid(True)
    exact = (np.exp(4*xx)-np.sinh(4.)*xx-np.cosh(4.))/16.
    plt.title(r'$\mathrm{max\ error=}$'+r'$'+str(np.linalg.norm(exact-uu,np.inf))+r'$', fontsize=14)
    plt.ylim([-2.5,0.5])
    plt.show()
interact(example_2,N=(2,35))

interactive(children=(IntSlider(value=16, description='N', max=35, min=2), Output()), _dom_classes=('widget-in…

<function __main__.example_2(N=16)>

# Example 3: Solving ODE 

Solving the following BVP $u_{xx}=\exp(u)$ with $u(-1)=u(1)=0$

In [96]:
def example_3(N=16,IT=20):
    D,x = cheb(N)
    D2  = np.dot(D,D)
    D2  = D2[1:-1,1:-1]

    u   = np.zeros(N-1)
    changes = np.zeros(IT)
    for i in np.arange(IT):
        u_new = np.linalg.solve(D2,np.exp(u))
        changes[i] = np.linalg.norm(u_new-u,np.inf)
        u = u_new

    u = np.concatenate(([0],u,[0]),axis=0)

    plt.figure(figsize=(2*M,M))
    plt.subplot(121)
    plt.plot(x,u,'k.')
    xx  = np.linspace(-1,1,1000)
    P   = np.polyfit(x, u, N)
    uu  = np.polyval(P, xx)
    plt.plot(xx,uu,'b-')
    plt.grid(True)
    plt.title(r'$\mathrm{IT=}$'+r'$'+str(IT)+r'$, $\mathrm{u(0)=}$'+r'$'+str(u[int(N/2)])+r'$', fontsize=14)
    plt.ylim([-0.5,0.])
    
    plt.subplot(122)
    plt.title(r'$\mathrm{Convergence}\, \left\|u^{(k+1)} - u^{(k)}\right\|_{\infty}$', fontsize=14, y=1.02)
    plt.semilogy(np.arange(IT),changes,'r.',markersize=12,markeredgewidth=3)
    plt.grid(True)
    plt.show()

interact(example_3,N=(2,30),IT=(0,100))

interactive(children=(IntSlider(value=16, description='N', max=30, min=2), IntSlider(value=20, description='IT…

<function __main__.example_3(N=16, IT=20)>

# Example 4: Eigenvalue BVP

Solve $-u_{xx}=\lambda\,u$ with $u(-1)=u(1)=0$

In [97]:
N_widget = IntSlider(min=2, max=50, step=1, value=10)
j_widget = IntSlider(min=1, max=49, step=1, value=5)

def update_j_range(*args):
    j_widget.max = N_widget.value-1
j_widget.observe(update_j_range, 'value')

def example_4(N=36,j=5):
    D,x = cheb(N)
    D2  = np.dot(D,D)
    D2  = D2[1:-1,1:-1]

    lam, V = np.linalg.eig(-D2)

    ii=np.argsort(np.real(lam))

    lam=lam[ii]
    V=V[:,ii]

    u = np.concatenate(([0],V[:,j-1],[0]),axis=0)

    plt.figure(figsize=(2*M,M))
    xx  = np.linspace(-1,1,1000)
    P   = np.polyfit(x, u, N)
    uu  = np.polyval(P, xx)
    plt.plot(xx,uu,'b-')
    plt.plot(x,u,'r.',markersize=10)
    plt.grid(True)
    plt.title(r'$\mathrm{eig}\,'+str(j)+'='+str(lam[j-1]*4./(np.pi**2))+r'\pi^2/4,\,'+r'\,\mathrm{ppw}=\,'+str(4*N/(np.pi*j))+r'$', fontsize=14)
    plt.show()
interact(example_4,N=N_widget,j=j_widget)

interactive(children=(IntSlider(value=10, description='N', max=50, min=2), IntSlider(value=5, description='j',…

<function __main__.example_4(N=36, j=5)>

# Example 5:  (2D) Poisson equation $u_{xx}+u_{yy}=f$ with u=0 on $\partial\Gamma$

In [106]:
elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)

def example_5(N=10,elev=40,azim=230,n_contours=8):
    D, x = cheb(N)
    y    = x
    D2   = np.dot(D,D)
    D2   = D2[1:-1,1:-1]

    xx_int, yy_int = np.meshgrid(x[1:-1],y[1:-1])
    xx_int_flat = xx_int.flatten('F')
    yy_int_flat = yy_int.flatten('F')

    f_lambda = lambda x, y: 10*np.sin(8*x*(y-1))

    f = f_lambda(xx_int_flat, yy_int_flat)

    I = np.eye(N-1)
    # The Laplacian
    L = np.kron(I,D2)+np.kron(D2,I)

    u = np.linalg.solve(L,f)

    fig = plt.figure(figsize=(2*M,2*M))

    # The spy of the Laplacian
    plt.subplot(221)
    plt.spy(L)

    # Plotting the approximation and its interpolation

    # The numerical approximation
    uu = np.zeros((N+1,N+1))
    uu[1:-1,1:-1]=np.reshape(u, (N-1,N-1), order='F')
    u_interp = sp.interpolate.RegularGridInterpolator((x, y), uu.T)
    
    xx_full, yy_full = np.meshgrid(x,y)

    plt.subplot(222, projection='3d')
    ax = fig.gca()
    ax.plot_wireframe(xx_full, yy_full, uu)
    ax.view_init(elev,azim)
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$y$') 

    # # The INTERPOLATED approximation

    N_fine=4*N
    x_finer_mesh = np.linspace(-1,1,N_fine)
    xx_finer_mesh, yy_finer_mesh = np.meshgrid(x_finer_mesh,x_finer_mesh)

    u_data_finer_mesh = u_interp((xx_finer_mesh,yy_finer_mesh))
    

    plt.subplot(224,projection='3d')
    ax = fig.gca()
    surf = ax.plot_surface(xx_finer_mesh, yy_finer_mesh, u_data_finer_mesh, rstride=1, cstride=1, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    fig.colorbar(surf)
    ax.view_init(elev,azim)
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
    
    plt.subplot(223)
    ax = fig.gca()
    plt.contour(xx_finer_mesh, yy_finer_mesh, u_data_finer_mesh, n_contours)
    plt.grid(True)
    plt.colorbar()
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
    
    plt.show()
interact(example_5,N=(3,40),elev=elev_widget,azim=azim_widget)

interactive(children=(IntSlider(value=10, description='N', max=40, min=3), IntSlider(value=40, description='el…

<function __main__.example_5(N=10, elev=40, azim=230, n_contours=8)>

# Example 6:  (2D) Helmholtz equation $u_{xx}+u_{yy}+k^2\,u=f$ with u=0 on $\partial\Gamma$

In [ ]:
elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)

def example_6(N=10,elev=40,azim=230,k=1,n_contours=8):
    D,x = cheb(N)
    y = x
    D2  = np.dot(D,D)
    D2  = D2[1:-1,1:-1]

    xx_int, yy_int = np.meshgrid(x[1:-1], y[1:-1])
    xx_int_flat = xx_int.flatten('F')
    yy_int_flat = yy_int.flatten('F')

    f_lambda = lambda x,y: np.exp(-10.*((y-1.)**2+(x-.5)**2))

    f = f_lambda(xx_int_flat, yy_int_flat)

    I = np.eye(N-1)
    # The Laplacian
    L = np.kron(I,D2)+np.kron(D2,I)+k**2*np.eye((N-1)**2)

    u = np.linalg.solve(L,f)

    fig = plt.figure(figsize=(2*M,2*M))

    # Plotting the approximation and its interpolation

    # The numerical approximation
    uu = np.zeros((N+1,N+1))
    uu[1:-1,1:-1] = np.reshape(u,(N-1,N-1), order='F')
    u_interp = sp.interpolate.RegularGridInterpolator((x, y), uu.T)
    
    xx_full, yy_full = np.meshgrid(x,y)

    plt.subplot(221,projection='3d')
    ax = fig.gca()
    ax.plot_wireframe(xx_full, yy_full, uu)
    ax.view_init(elev,azim)

    plt.subplot(222)
    ax = fig.gca()
    plt.contour(xx_full, yy_full, uu, n_contours)
    plt.grid(True)
    plt.colorbar()
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
   
    # The INTERPOLATED approximation
    N_fine=4*N
    finer_mesh = np.linspace(-1,1,N_fine)
    xx_finer_mesh, yy_finer_mesh = np.meshgrid(finer_mesh,finer_mesh)

    u_data_finer_mesh = u_interp((xx_finer_mesh,yy_finer_mesh))
    
    plt.subplot(223,projection='3d')
    ax = fig.gca()
    ax.plot_wireframe(xx_finer_mesh, yy_finer_mesh, u_data_finer_mesh)
    ax.view_init(elev,azim)
    
    plt.subplot(224)
    ax = fig.gca()
    plt.contour(xx_finer_mesh, yy_finer_mesh, u_data_finer_mesh, n_contours)
    plt.grid(True)
    plt.colorbar()
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
    
    plt.show()
interact(example_6,N=(3,40),elev=elev_widget,azim=azim_widget,k=(1,20),n_contours=(5,12))

interactive(children=(IntSlider(value=10, description='N', max=40, min=3), IntSlider(value=40, description='el…

<function __main__.example_6(N=10, elev=40, azim=230, k=1, n_contours=8)>

# Example 7:  (2D)  $-(u_{xx}+u_{yy})=\lambda\,u$ with u=0 on $\partial\Gamma$

In [100]:
elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)
N_widget = IntSlider(min=2, max=30, step=1, value=10)
j_widget = IntSlider(min=1, max=20, step=1, value=1)

def update_j_range(*args):
    j_widget.max = (N_widget.value-1)**2
j_widget.observe(update_j_range, 'value')

def example_7(N=10,elev=40,azim=230,n_contours=8,j=1):
    D,x = cheb(N)
    y=x
    D2  = np.dot(D,D)
    D2  = D2[1:-1,1:-1]

    xx_int, yy_int=np.meshgrid(x[1:-1],y[1:-1])
    xx_int_flat = xx_int.flatten('F')
    yy_int_flat = yy_int.flatten('F')

    I = np.eye(N-1)
    # The Laplacian
    L = (np.kron(I,-D2)+np.kron(-D2,I))

    # Computing eigenvalues and eigenvectors, where the eigenvectors approximate the eigenfunctions!
    lam, V = np.linalg.eig(L)

    ii=np.argsort(np.real(lam))
    lam=lam[ii]
    V=V[:,ii]

    fig = plt.figure(figsize=(2*M,M))

    # Plotting the approximation and its interpolation

    # The numerical approximation
    vv = np.zeros((N+1,N+1))
    vv[1:-1,1:-1]=np.reshape(np.real(V[:,j-1]),(N-1,N-1), order='F')
    u_interp = sp.interpolate.RegularGridInterpolator((x, y), vv)
    
    xx_full, yy_full = np.meshgrid(x,y)

    plt.subplot(221,projection='3d')
    ax = fig.gca()
    ax.plot_wireframe(xx_full, yy_full, vv)
    plt.title(r'$\lambda_'+str(j)+'$'+r'$/(\pi/2)^2= '+str(lam[j-1]/((np.pi/2)**2))+r'$', fontsize=14)
    ax.view_init(elev,azim)

    plt.subplot(222)
    ax = fig.gca()
    plt.contour(xx_full, yy_full, vv, n_contours)
    plt.grid(True)
    plt.colorbar()
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')

    # The INTERPOLATED approximation
    N_fine=4*N
    x_finer_mesh=np.linspace(-1,1,N_fine)
    xx_finer_mesh,yy_finer_mesh=np.meshgrid(x_finer_mesh,x_finer_mesh)
    v_interp = sp.interpolate.RegularGridInterpolator((x, y), vv.T, method='linear')
    v_data_finer_mesh=v_interp((xx_finer_mesh,yy_finer_mesh))

    plt.subplot(223,projection='3d')
    ax = fig.gca()
    ax.plot_wireframe(xx_finer_mesh, yy_finer_mesh, v_data_finer_mesh)
    ax.view_init(elev,azim)

    plt.subplot(224)
    ax = fig.gca()
    plt.contour(xx_finer_mesh, yy_finer_mesh, v_data_finer_mesh, n_contours)
    plt.grid(True)
    plt.colorbar()
    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
    

    plt.show()
interact(example_7,N=N_widget,elev=elev_widget,azim=azim_widget,n_contours=(5,12),j=j_widget)

interactive(children=(IntSlider(value=10, description='N', max=30, min=2), IntSlider(value=40, description='el…

<function __main__.example_7(N=10, elev=40, azim=230, n_contours=8, j=1)>

# In-class work

## [Flash back] Implement Program 6, 7 and 12.

## [Today] Implement Program 19, 20, 21, 22 and 23.